In [1]:
import Pkg; Pkg.activate(".")

Activating environment at `~/.julia/dev/VHEM/notebooks/Project.toml`


In [2]:
using Revise
using BenchmarkTools
using Distributions
using HMMBase
using VHEM

┌ Info: Precompiling VHEM [be869012-f1ee-4d1b-ac0c-66b7016f7c90]
└ @ Base loading.jl:1273
┌ Warning: Package VHEM does not have ArgCheck in its dependencies:
│ - If you have VHEM checked out for development and have
│   added ArgCheck as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with VHEM
└ Loading ArgCheck into VHEM from project dependency, future warnings for VHEM are suppressed.


In [3]:
import Base: rand

rand(::Type{Normal}) = Normal(rand(Normal(0, 10)), rand(Gamma(1, 2)))

function rand(::Type{MixtureModel})
    K = rand(1:10)
    p = rand(Dirichlet(K, 1.0))
    c = [rand(Normal) for _ in 1:K]
    MixtureModel(c, p)
end

function rand(::Type{HMM})
    K = rand(1:10)
    a = rand(Dirichlet(K, 1.0))
    A = randtransmat(K, 1.0)
    B = [rand(MixtureModel) for _ in 1:K]
    HMM(a, A, B)
end

rand (generic function with 179 methods)

In [139]:
hmm1 = rand(HMM)
hmm2 = rand(HMM)#HMM([0.99 0.01; 0.01 0.99], [MixtureModel([Normal(0,1),Normal(0,2)]), MixtureModel([Normal(10,1),Normal(10,2)])])
hmm3 = rand(HMM)
base = H3M([hmm1], [1.0])
redu = H3M([hmm1,hmm3], [0.5, 0.5]);

In [83]:
lgmm = zeros(size(hmm1,1), size(hmm2,1))
@btime for i in 1:size(hmm1,1), j in 1:size(hmm2,1)
     lgmm[i,j] = lowerbound(hmm1.B[i], hmm2.B[j])[3]
end
lgmm

  87.451 μs (703 allocations: 88.50 KiB)


8×7 Array{Float64,2}:
 -3.53102   -3.29874   -5.58802   -3.54779   -7.39157   -4.4917    -4.54644
 -8.18694   -6.78814  -10.0176    -5.94913  -13.1523   -10.0887   -13.9068 
 -4.27602   -4.24909   -6.33353   -4.12885  -10.8714    -6.53162   -3.96025
 -3.60089   -4.91482   -5.04518   -4.75427   -6.11385   -6.90928   -3.89487
 -4.72921   -5.18436   -4.22306   -4.29905   -8.54102   -7.0201    -6.61181
 -4.8938   -14.3924   -11.8339   -11.6958    -4.29281   -4.78074  -15.4633 
 -3.6992    -3.71138   -8.91805   -3.46717   -3.59427   -2.264     -5.64321
 -3.23139   -4.19019   -4.83167   -4.66577   -8.87759   -5.48787   -4.32292

In [80]:
@btime lowerbound(hmm1, hmm2, lgmm, 20);

  224.804 μs (2152 allocations: 373.38 KiB)


In [65]:
@btime vhem_step(base, redu, 20, 10);

  1.338 ms (42008 allocations: 1.30 MiB)


In [150]:
@btime vhem_step(base, redu, 20, 10);

  823.997 μs (25679 allocations: 878.02 KiB)


In [127]:
length(redu)

2

In [146]:
vhem_step(base, redu, 20, 10)[1].M[2].A

3×3 Array{Float64,2}:
 0.211083   0.00341476  0.785503
 0.108014   0.286909    0.605077
 0.0494516  0.736479    0.214069

In [149]:
vhem_step(base, redu, 20, 10)[1].M[1].A

3×3 Array{Float64,2}:
 0.677344  0.0458447  0.276811
 0.193439  0.701696   0.104865
 0.243106  0.546097   0.210797

In [64]:
@code_warntype vhem_step(base, redu, 20, 10)

Variables
  #self#::Core.Compiler.Const(VHEM.vhem_step, false)
  base::H3M{HMM{Univariate,Float64}}
  reduced::H3M{HMM{Univariate,Float64}}
  τ::Int64
  N::Int64
  logη::Dict{Tuple{Int64,Int64},Dict{Tuple{Int64,Int64},Array{Float64,2}}}
  lgmm::Dict{Tuple{Int64,Int64},Array{Float64,2}}
  lhmm::Array{Float64,2}
  logν::Dict{Tuple{Int64,Int64},Array{Float64,3}}
  logξ::Dict{Tuple{Int64,Int64},Array{Float64,4}}
  νagg1::Dict{Tuple{Int64,Int64},Array{Float64,1}}
  νagg::Dict{Tuple{Int64,Int64},Array{Float64,2}}
  ξagg::Dict{Tuple{Int64,Int64},Array{Float64,2}}
  @_14::Union{Nothing, Tuple{Tuple{Int64,HMM{Univariate,Float64}},Tuple{Int64,Int64}}}
  logz::Array{Float64,2}
  @_16::Union{Nothing, Tuple{Tuple{Int64,Float64},Tuple{Int64,Int64}}}
  z::Array{Float64,2}
  newω::Array{Float64,1}
  newM::Array{HMM,1}
  @_20::Union{Nothing, Tuple{Tuple{Int64,Tuple{HMM{Univariate,Float64},Float64}},Tuple{Int64,Tuple{Int64,Int64}}}}
  i@_21::Int64
  Mi::HMM{Univariate,Float64}
  @_23::Int64
  @_24::Unio

In [16]:
@btime vhem_step(base, redu, 20, 10);

  6.668 ms (118910 allocations: 3.68 MiB)


In [29]:
vhem_step(base, redu, 20, 10);

sum(νagg[i, j]) = 19.99999999999996
sum(ξagg[i, j]) = 215.0


In [77]:
@btime lowerbound(hmm1, hmm2, lgmm, 20);

  227.385 μs (2074 allocations: 409.48 KiB)


In [167]:
lowerbound(hmm1, hmm2, lgmm, 20)[3]

norm = 7.203107252446733e21
logϕ1[β, :] = [7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21, 7.203107252446733e21]
norm = 7.648960865928461e21
logϕ1[β, :] = [7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21, 7.648960865928461e21]
norm = 6.429940704891407e21
logϕ1[β, :] = [6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21, 6.429940704891407e21]
norm = 6.751360294546391e21
logϕ1[β, :] = [6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21, 6.751360294546391e21]
norm = 7.147951370984953e21
logϕ1[β,

6×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [58]:
lowerbound(hmm1, hmm2, lgmm, 20)[1]

-120.53574888682752

In [72]:
lowerbound(hmm1, hmm2, lgmm, 20)[10,3,2,4]

-30.327392578125

In [70]:
lowerbound(hmm1, hmm2, lgmm, 20)[2][10,2,3,4]

BoundsError: BoundsError

In [56]:
@code_warntype lowerbound(hmm1.B[1], hmm2.B[1])

Variables
  #self#::Core.Compiler.Const(VHEM.lowerbound, false)
  a::MixtureModel{Univariate,Continuous,Normal{Float64},Float64}
  b::MixtureModel{Univariate,Continuous,Normal{Float64},Float64}
  I::Int64
  J::Int64
  lb::Float64
  logl::Array{Float64,2}
  logη::Array{Float64,2}
  logp::Array{Float64,1}
  @_10::Union{Nothing, Tuple{Tuple{Int64,Normal{Float64}},Tuple{Int64,Int64}}}
  @_11::Union{Nothing, Tuple{Int64,Int64}}
  i@_12::Int64
  ai::Normal{Float64}
  @_14::Int64
  @_15::Union{Nothing, Tuple{Tuple{Int64,Normal{Float64}},Tuple{Int64,Int64}}}
  j@_16::Int64
  bj::Normal{Float64}
  @_18::Int64
  p::Float64
  @_20::Union{Nothing, Tuple{Int64,Int64}}
  i@_21::Int64
  j@_22::Int64

Body::Tuple{Array{Float64,2},Array{Float64,2},Float64}
1 ──        Core.NewvarNode(:(@_11))
│    %2   = VHEM.ncomponents(a)::Int64
│    %3   = VHEM.ncomponents(b)::Int64
│           (I = %2)
│           (J = %3)
│           (lb = 0.0)
│    %7   = Core.apply_type(VHEM.Matrix, VHEM.Float64)::Core.Compiler.

In [7]:
hmm2.B[1]

MixtureModel{Normal{Float64}}(K = 5)
components[1] (prior = 0.2625): Normal{Float64}(μ=-8.217962586823678, σ=0.11937310975407141)
components[2] (prior = 0.1975): Normal{Float64}(μ=-7.064906464952637, σ=0.7910233039497935)
components[3] (prior = 0.0737): Normal{Float64}(μ=12.009901987699267, σ=0.9060441313999246)
components[4] (prior = 0.4606): Normal{Float64}(μ=10.30102621680343, σ=6.386081022761737)
components[5] (prior = 0.0057): Normal{Float64}(μ=-14.418388427464933, σ=0.36107550718870646)
